In [4]:
cwltool workflow.cwl --stac_item https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A --asset_id visual

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'workflow.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/03%20Capture%20JSON%20and%20reuse/workflow.cwl'
INFO [workflow ] start
INFO [workflow ] starting step step_stac
INFO [step step_stac] start
INFO [job step_stac] /tmp/s_c4v8st$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/s_c4v8st,target=/AkNkpv \
    --mount=type=bind,source=/tmp/u1v8roae,target=/tmp \
    --workdir=/AkNkpv \
    --read-only=true \
    --log-driver=none \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/aix5x2yn/20240402141905-932712.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/AkNkpv \
    docker.io/curlimages/curl:latest \
    curl \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A > /tmp/s_c4v8st/message
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   S

In [5]:
cat quicklook.png | display